# Objective

This notebook shows the connection to AWS and a Hello World with some of the services used

1) S3
2) Lambda (try the hello world from midterm)
3) Cloud Watch

To connect a key was made using IAM, and put into a local `.env` file, the credentials are temporary and will expire in 30 days. Care should be used when making these and different keys used for dev and prod.


* outside POC environment consider different security practices such as SSO 

## TODO
* TODO: get out metrics from runs\, now that we can download them, pipe into useable place
* TODO: try a lambda function hello world, show some utility
* TODO: check that we are using the cloudwatch correctly generally (check book)


In [1]:
import os
import sys

import boto3
import pandas as pd
from tqdm import tqdm
import yaml


print(sys.version)

with open("../keys/aws_credentials.yaml", "r") as f:
    credentials = yaml.safe_load(f)

3.11.8 (v3.11.8:db85d51d3e, Feb  6 2024, 18:02:37) [Clang 13.0.0 (clang-1300.0.29.30)]


### Initialize Services

In [2]:
# Initialize  -- using session authorization
session = boto3.Session(aws_access_key_id=credentials["aws_access_key_id"],
                        aws_secret_access_key=credentials["aws_secret_access_key"],
                        region_name=credentials["region"])

s3 = session.client('s3')
lamda_func = session.client("lambda")
cloudwatch = session.client('logs')

### Test S3 Connection

In [3]:
d = s3.list_buckets()

# show current buckets
b = [n["Name"] for n in d["Buckets"]]


# buckets needed
assert 'fmi-lambda-demo' in b, "missing the lambda demo" # midterm
assert 'team4-cosmicai' in b, "missing our team4 cosmicai S3 connection" 

b

['aws-athena-query-results-211125778552-us-east-1',
 'cosmicai-data',
 'cosmicai2',
 'fmi-lambda-demo',
 'group2-s3-bucket',
 'group4-s3-bucket',
 'sagemaker-studio-211125778552-3zpozdpwzcx',
 'sagemaker-studio-211125778552-rrp76qgcj1n',
 'sagemaker-us-east-1-211125778552',
 'team-one-cosmic-data',
 'team-one-s3-cosmic',
 'team2cosmicai',
 'team3cosmicai',
 'team4-cosmicai']

### Test Download S3 buckets

* useful for later, consider putting in a src or util script

In [4]:

def download_s3_bucket(s3, bucket_name :str, local_dir:str = "tmp") -> None:
    """takes an S3 object, and a valid bucket name and downloads all the files on that Bucket
    in the same structure and copies them to a local directory.
    
    PARAMS:
        s3: a botocore s3 object
        bucket_name: a valid s3 bucket in that object
        local_directory: where the bucket will get copied to

    for fine grained control use `s3.download_file` and for a list of valid buckets `s3.list_buckets`.
    If local directory not specified will dump into tmp/ where this is script is called. Files downloaded
    should have the same structure as the S3 bucket.  
    """

    # Ensure the local directory exists
    if not os.path.exists(local_dir):
        print(f"creating directory -- {local_dir}")
        os.makedirs(local_dir)

    # List objects in the specified S3 bucket
    objects = s3.list_objects_v2(Bucket=bucket_name)

    if 'Contents' in objects:
        # Initialize tqdm progress bar
        total_files = len(objects['Contents'])
        with tqdm(total=total_files, desc="Downloading files", unit="file") as pbar:
            for obj in objects['Contents']:

                local_file_path = os.path.join(local_dir, obj['Key'])
                
                # Ensure the directory structure exists
                if not os.path.exists(os.path.dirname(local_file_path)):
                    os.makedirs(os.path.dirname(local_file_path))
                
                # Check if the object is a file (not a directory)
                if not obj['Key'].endswith('/'):
                    s3.download_file(bucket_name, obj['Key'], local_file_path)
                
                # Update the progress bar
                pbar.update(1)
        print("Download complete.")
    else:
        print(f"No objects found in bucket {bucket_name}.")



In [5]:
# Example usage, run only if needed
BUCKET_NAME = "team4-cosmicai"
LOCAL_DIR = '..//data//raw//test'
# download_s3_bucket(s3, BUCKET_NAME, LOCAL_DIR)

In [6]:
# check that it ran ok
!tree ..//data//raw//test

..//data//raw//test
├── payload.json
├── results
│   ├── 0
│   │   ├── Results.json
│   │   └── ResultsReduce.json
│   └── 1
├── scripts
│   └── Anomaly Detection
│       ├── Fine_Tune_Model
│       │   └── Mixed_Inception_z_VITAE_Base_Img_Full_New_Full.pt
│       ├── Inference
│       │   ├── __init__.py
│       │   ├── fmilib
│       │   │   ├── __init__.py
│       │   │   ├── fmi_operations.py
│       │   │   └── fmi_scaling_lambda.py
│       │   ├── inference.py
│       │   └── resized_inference.pt
│       ├── NormalCell.py
│       ├── Plot_Redshift.py
│       ├── Plots
│       ├── __init__.py
│       └── blocks
│           ├── concat_data.py
│           ├── model_vit_inception.py
│           ├── photoz.py
│           └── split_data.py
└── scripts_ssd
    └── Anomaly Detection
        ├── Astronomy_Overview.pptx
        ├── Fine_Tune_Model
        │   └── Mixed_Inception_z_VITAE_Base_Img_Full_New_Full.pt
        ├── Inference
        │   ├── Inference Step by Step Instructions.pdf


## Lambda Functions
***

* This is a place holder for the Lambda functions. 

## Log Data
***

To pull log events, the stream is needed, to pull the stream the group is needed, broadly

```mermaid
graph TD
    A[Log Groups] --> B[Log Streams]
    B --> C[Log Events]
```

***

### Log Groups


In [7]:

l = []
r = cloudwatch.describe_log_groups()

for group in r['logGroups']:
     l.append(group['logGroupName'])

df_log_groups = pd.DataFrame(l, columns=["log_group_names"])

# general if needed
df_log_groups[df_log_groups.log_group_names.str.contains("(?!.*sagemaker).*")].head(25) # don't include sagemaker, many instances related to labs

# may be interested in the cosmic ai logs
# df_log_groups[df_log_groups.log_group_names.str.contains("cosmic")]


,log_group_names
0,/aws-glue/column-statistics
1,/aws-glue/jobs/error
2,/aws-glue/jobs/output
3,/aws-glue/sessions/error
4,/aws-glue/sessions/output
5,/aws-glue/testconnection/error/Redo
6,/aws-glue/testconnection/error/Redshift_connec...
7,/aws-glue/testconnection/error/team3-con
8,/aws-glue/testconnection/output/Redo
9,/aws-glue/testconnection/output/Redshift_conne...


### Log Streams
***

In [8]:
LOG_GROUP = "/aws/lambda/data-parallel-init"

l = []

r = cloudwatch.describe_log_streams(logGroupName=LOG_GROUP)
for stream in r['logStreams']:
    l.append(stream['logStreamName'])

df_log_streams_raw = pd.DataFrame(l, columns=["raw_streams"])

In [9]:
# what a stream name should look like
df_log_streams_raw.iloc[-10].values[0]

'2024/11/22/[$LATEST]272db91f41c1428e988f3084f113f535'

In [10]:
# to make more readable 

df_log_streams = df_log_streams_raw["raw_streams"].str.split(r"\[\$LATEST\]", expand=True)
df_log_streams.columns = ["date_pulled", "stream_hash"]

df_log_streams

,date_pulled,stream_hash
0,2024/11/07/,0fcf0e06f1174a16aca0adba257efc84
1,2024/11/07/,1738a628c5cf4796b51d6c481da4b746
2,2024/11/07/,27d9c7e9e6464016b8dfdd647c214fb5
3,2024/11/07/,6a1b2157e1c6436db3bdc4e740fd2f41
4,2024/11/20/,0d836483f503404fa3ad3d332da2f8c4
5,2024/11/20/,344f2b0cdf904d4a9dce3aba4e7158e3
6,2024/11/20/,586ddf9995cd4000a65778d5853521c8
7,2024/11/22/,0eaeff78011e4031a381ecc431f1b26a
8,2024/11/22/,272db91f41c1428e988f3084f113f535
9,2024/11/22/,3dd38b8393f848bfaff52e67b369a996


### Log Events
***

In [11]:
# can now get log events

# try the latest (already in order oldest -> newest or desc)
LOG_STREAM = df_log_streams_raw.iloc[-1].values[0]

r = cloudwatch.get_log_events(logGroupName=LOG_GROUP, logStreamName=LOG_STREAM)

for event in r['events']:
    print(f"Timestamp: {event['timestamp']}, Message: {event['message']}")

Timestamp: 1732638297021, Message: INIT_START Runtime Version: python:3.12.v38	Runtime Version ARN: arn:aws:lambda:us-east-1::runtime:7515e00d6763496e7a147ffa395ef5b0f0c1ffd6064130abb5ecde5a6d630e86

Timestamp: 1732638297359, Message: [INFO]	2024-11-26T16:24:57.359Z		Found credentials in environment variables.

Timestamp: 1732638297567, Message: START RequestId: a7f629b6-8625-4d16-96e9-f20fbf8b9f5e Version: $LATEST

Timestamp: 1732638299159, Message: END RequestId: a7f629b6-8625-4d16-96e9-f20fbf8b9f5e

Timestamp: 1732638299159, Message: REPORT RequestId: a7f629b6-8625-4d16-96e9-f20fbf8b9f5e	Duration: 1591.85 ms	Billed Duration: 1592 ms	Memory Size: 128 MB	Max Memory Used: 84 MB	Init Duration: 543.20 ms	



In [12]:
event

{'timestamp': 1732638299159,
 'message': 'REPORT RequestId: a7f629b6-8625-4d16-96e9-f20fbf8b9f5e\tDuration: 1591.85 ms\tBilled Duration: 1592 ms\tMemory Size: 128 MB\tMax Memory Used: 84 MB\tInit Duration: 543.20 ms\t\n',
 'ingestionTime': 1732638301093}

In [13]:
pd.json_normalize(r, record_path=["events"])

,timestamp,message,ingestionTime
0,1732638297021,INIT_START Runtime Version: python:3.12.v38\tR...,1732638301093
1,1732638297359,[INFO]\t2024-11-26T16:24:57.359Z\t\tFound cred...,1732638301093
2,1732638297567,START RequestId: a7f629b6-8625-4d16-96e9-f20fb...,1732638301093
3,1732638299159,END RequestId: a7f629b6-8625-4d16-96e9-f20fbf8...,1732638301093
4,1732638299159,REPORT RequestId: a7f629b6-8625-4d16-96e9-f20f...,1732638301093


In [ ]:
## TODO: use regex to parse log streams?